# Dependencias

In [1]:
%pip install opencv-contrib-python
%pip install plotly
import cv2
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import scipy.signal as signal
from scipy.optimize import curve_fit

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Definiciones

In [2]:
# Carga el video
nombre_video = 'yoyoyo.MOV'
cap = cv2.VideoCapture('Videos/' + nombre_video) # Local
if not cap.isOpened():
  cap = cv2.VideoCapture('/content/' + nombre_video) # Colab


# Medidas tomadas de pixeles y metros
medida_metros = 0.96
# Cantidad de pixeles en 0.96 metros
medida_pixeles = 1593
#Masa del yoyo (sin hilo) en kg
masa_yoyo = 0.02235
#Masa del hilo en kg
masa_hilo = 0.0661

#Define los rectangulos a trackear 
bboxCentro = (516, 302, 88, 82)
bboxBorde = (537, 344, 47, 51)

#Constante de conversion px a metros
const_XY = medida_metros/medida_pixeles

# Variables para guardar las posiciones y tiempos
positionsCentro = []
positionsCentro_m = []
positionsBorde = []
positionsBorde_m = []
fps = cap.get(cv2.CAP_PROP_FPS)  # Obtiene los FPS del video
frame_duration = 1.0 / fps  # Duración de cada frame en segundos # Si hay error acá, revisar video
frame_count = 0

# Trackeo

In [3]:
def calcularPosiciones(frame, bbox, positions, positions_m, number):
    # Calcular centro
    (x, y, w, h) = [int(v) for v in bbox]
    center_x = x + w // 2
    center_y = y + h // 2
    # Dibujar bounding box
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # Dibuja centro
    cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)

    # Guarda posiciones
    current_time = frame_count * frame_duration
    positions.append((current_time, center_x, cap.get(cv2.CAP_PROP_FRAME_HEIGHT) - center_y))
    positions_m.append((current_time, center_x * const_XY, (cap.get(cv2.CAP_PROP_FRAME_HEIGHT) - center_y) * const_XY))

In [4]:
# Inicializar bounding boxes
bbox1 = bboxCentro
bbox2 = bboxBorde
# Crear trackers
trackerCentro = cv2.TrackerCSRT_create()
trackerBorde = cv2.TrackerCSRT_create()
# Leer primer frame
ret, frame = cap.read()
if not ret:
    print("No se pudo leer el video.")
    cap.release()
    cv2.destroyAllWindows()
# Inicializar trackers
ok1 = trackerCentro.init(frame, bbox1) # Si hay error acá, compilar definiciones
ok2 = trackerBorde.init(frame, bbox2)

while True:
    # Lee el siguiente frame
    ret, frame = cap.read()

    if not ret:
        break

    # Actualiza ambos trackers
    ok1, bbox1 = trackerCentro.update(frame)
    ok2, bbox2 = trackerBorde.update(frame)
    if ok1 and ok2:
        calcularPosiciones(frame, bbox1, positionsCentro, positionsCentro_m, 1)
        calcularPosiciones(frame, bbox2, positionsBorde, positionsBorde_m, 2)

    # Muestra el frame con el seguimiento SOLO LOCAL, NO COLAB
    #display_frame = cv2.resize(frame, (400, 1000))
    #cv2.imshow('Object Tracking', display_frame)

    # Rompe el bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Incrementa el contador de frames
    frame_count += 1

# Libera la captura y cierra las ventanas
cap.release()
cv2.destroyAllWindows()

# Datos

In [5]:
#Función para pasar de coordenadas cartesianas a polares

def cartesian_to_polar(x_coords, y_coords):
    '''
    Esta función devuelve una lista con el radio r y otra lista con el ángulo theta medido desde la horizontal para las coordenadas polares a
    partir de las coordenadas cartesianas.
    x: lista con los valores de x
    y: lista con los valores de y
    '''
    r_coords = [] # Inicializo una lista vacía para los valores de la coordenada r
    theta_coords = [] # Inicializo una lista vacía para los valores del ángulo medido desde el eje +x
    for x, y in zip(x_coords, y_coords):
        r = np.sqrt(x**2 + y**2)
        theta = np.arctan2(y, x)
        r_coords.append(r)
        theta_coords.append(theta)

    return r_coords, theta_coords

In [6]:
suavizado = True

def crear_dataframe(positions):
    # Crea el DataFrame y aplica filtros
    df = pd.DataFrame(positions, columns=['Time (sec)', 'X', 'Y'])

    if suavizado:
      window_size = 13
      polynomial_order = 2
      df['X'] = signal.savgol_filter(df['X'], window_size, polynomial_order)
      df['Y'] = signal.savgol_filter(df['Y'], window_size, polynomial_order)

    diff_size = 3

    # Calcular coordenadas polares
    df['R'], df['Theta'] = cartesian_to_polar(df['X'], df['Y'])  # Agregar coordenadas polares

    # Continuar con el resto del cálculo
    df['Delta_Time'] = df['Time (sec)'].diff().fillna(0)
    df['Delta_X'] = df['X'].diff(diff_size).fillna(0)
    df['Delta_Y'] = df['Y'].diff(diff_size).fillna(0)

    # Velocidades
    df['Speed_X'] = df['Delta_X'] / df['Delta_Time']
    df.loc[0, 'Speed_X'] = df.loc[1, 'Speed_X']
    df['Speed_Y'] = df['Delta_Y'] / df['Delta_Time']
    df.loc[0, 'Speed_Y'] = df.loc[1, 'Speed_Y']
    df['Speed_X'] = df['Speed_X'].replace([np.inf, -np.inf], 0)
    df['Speed_Y'] = df['Speed_Y'].replace([np.inf, -np.inf], 0)
    df['Speed'] = np.sqrt(df['Speed_X']**2 + df['Speed_Y']**2)
    df['Speed'] = df['Speed'].replace([np.inf, -np.inf], 0)

    # Savitzky-Golay filters para suavizar las velocidades
    if suavizado:
      df['Speed_X'] = signal.savgol_filter(df['Speed_X'], window_size, polynomial_order)
      df['Speed_Y'] = signal.savgol_filter(df['Speed_Y'], window_size, polynomial_order)
      df['Speed'] = signal.savgol_filter(df['Speed'], window_size, polynomial_order)

    # Calcular aceleraciones
    df['Acceleration_X'] = df['Speed_X'].diff(diff_size) / df['Delta_Time']
    df['Acceleration_X'] = df['Acceleration_X'].fillna(0)
    df['Acceleration_Y'] = df['Speed_Y'].diff(diff_size) / df['Delta_Time']
    df['Acceleration_Y'] = df['Acceleration_Y'].fillna(0)
    df['Acceleration_X'] = df['Acceleration_X'].replace([np.inf, -np.inf], 0)
    df['Acceleration_Y'] = df['Acceleration_Y'].replace([np.inf, -np.inf], 0)
    df['Acceleration'] = np.sqrt(df['Acceleration_X']**2 + df['Acceleration_Y']**2)
    df['Acceleration'] = df['Acceleration'].replace([np.inf, -np.inf], 0)

    # Cálculo de la velocidad angular
    df['Delta_Theta'] = df['Theta'].diff().fillna(0)  # Cambio en el ángulo
    df['Angular_Velocity'] = df['Delta_Theta'] / df['Delta_Time']  # Velocidad angular en radianes por segundo
    df['Angular_Velocity'] = df['Angular_Velocity'].replace([np.inf, -np.inf], 0)  # Eliminar infinitos
    if suavizado:
      df['Angular_Velocity'] = signal.savgol_filter(df['Angular_Velocity'], window_size, polynomial_order)

    #Cálculo de la aceleración angular
    df['Delta_Omega']= df['Angular_Velocity'].diff().fillna(0)
    df['Angular_Acceleration'] = df['Delta_Omega'] / df['Delta_Time']
    #df['Angular_Acceleration'] = df['Angular_Acceleration'].replace([np.inf, -np.inf], 0)
    if suavizado:
      df['Angular_Acceleration'] = signal.savgol_filter(df['Angular_Acceleration'], window_size, polynomial_order)

    # Separar dataframes para velocidades y aceleraciones
    df_speed = df[['Time (sec)', 'Speed_X', 'Speed_Y', 'Speed']][df['Speed'] != 0].copy()
    df_acceleration = df[['Time (sec)', 'Acceleration_X', 'Acceleration_Y', 'Acceleration']][df['Acceleration'] != 0].copy()

    return df, df_speed, df_acceleration




def crear_dataframeRelativo(dfA, dfB):
    dfCombined = pd.merge(dfA, dfB, on='Time (sec)')
    dfCombined['Delta_Time'] = dfCombined['Time (sec)'].diff().fillna(0)

    # Posición relativa en X e Y
    dfCombined['Relative_Position_X'] = dfCombined['X_x'] - dfCombined['X_y']
    dfCombined['Relative_Position_Y'] = dfCombined['Y_x'] - dfCombined['Y_y']

    # Calcular coordenadas polares de la posición relativa
    dfCombined['R_Relative'], dfCombined['Theta_Relative'] = cartesian_to_polar(
        dfCombined['Relative_Position_X'], dfCombined['Relative_Position_Y']
    )

    dfCombined['Theta_Relative'] = np.unwrap(dfCombined['Theta_Relative'])
    dfCombined.loc[dfCombined['Delta_Time'] <= 1e-5, 'Angular_Velocity_Relative'] = 0

    # Calcular la velocidad angular relativa, manejando posibles discontinuidades
    dfCombined['Delta_Theta'] = dfCombined['Theta_Relative'].diff().fillna(0)
    dfCombined['Relative_Angular_Velocity'] = dfCombined['Delta_Theta'] / dfCombined['Delta_Time']
    dfCombined['Relative_Angular_Velocity'] = dfCombined['Relative_Angular_Velocity'].replace([np.inf, -np.inf], 0)

    #Aceleración angular relativa
    dfCombined['Delta_Relative_Omega'] = dfCombined['Relative_Angular_Velocity'].diff().fillna(0)
    dfCombined['Relative_Angular_Acceleration'] = dfCombined['Delta_Relative_Omega'] / dfCombined['Delta_Time']
    if suavizado:
      dfCombined['Relative_Angular_Acceleration'] = signal.savgol_filter(dfCombined['Relative_Angular_Acceleration'], 26, 2)

    # Calcular velocidades y aceleraciones relativas
    dfCombined['Delta_X'] = dfCombined['Relative_Position_X'].diff().fillna(0)
    dfCombined['Delta_Y'] = dfCombined['Relative_Position_Y'].diff().fillna(0)
    dfCombined['Relative_Speed_X'] = dfCombined['Delta_X'] / dfCombined['Delta_Time']
    dfCombined['Relative_Speed_Y'] = dfCombined['Delta_Y'] / dfCombined['Delta_Time']
    if suavizado:
      dfCombined['Relative_Speed_X'] = signal.savgol_filter(dfCombined['Relative_Speed_X'], 26, 2)
      dfCombined['Relative_Speed_Y'] = signal.savgol_filter(dfCombined['Relative_Speed_Y'], 26, 2)

    dfCombined['Relative_Acceleration_X'] = dfCombined['Relative_Speed_X'].diff() / dfCombined['Delta_Time']
    dfCombined['Relative_Acceleration_Y'] = dfCombined['Relative_Speed_Y'].diff() / dfCombined['Delta_Time']
    if suavizado:
      dfCombined['Relative_Acceleration_X'] = signal.savgol_filter(dfCombined['Relative_Acceleration_X'], 26, 2)
      dfCombined['Relative_Acceleration_Y'] = signal.savgol_filter(dfCombined['Relative_Acceleration_Y'], 26, 2)

    # Crear dataframes de velocidad y aceleración relativas
    dfCombined_speed = dfCombined[['Time (sec)', 'Relative_Speed_X', 'Relative_Speed_Y']]
    dfCombined_acceleration = dfCombined[['Time (sec)', 'Relative_Acceleration_X', 'Relative_Acceleration_Y']]

    return dfCombined, dfCombined_speed, dfCombined_acceleration


In [7]:
# Creamos los DataFrames para los puntos
dfCentro, dfCentro_speed, dfCentro_acceleration = crear_dataframe(positionsCentro_m)
dfBorde, dfBorde_speed, dfBorde_acceleration = crear_dataframe(positionsBorde_m)
dfRelativoBordeCentro, dfRelativoBordeCentro_speed, dfRelativoBordeCentro_acceleration = crear_dataframeRelativo(dfBorde, dfCentro)

# Guardamos los DataFrames actualizados en archivos CSV (es para chequear después si todo está en orden)
dfCentro.to_csv('DataFrameA.csv', index=False, float_format='%.6f')
dfBorde.to_csv('DataFrameB.csv', index=False, float_format='%.6f')
dfRelativoBordeCentro.to_csv('DataFrameC.csv', index=False, float_format='%.6f')
print("Datos de posiciones, velocidades y velocidad angular relativa guardados en archivos CSV.")

# Comprobación opcional de conteo de filas
#print(dfCentro_acceleration.count())
#print(dfCentro_speed.count())
#print(dfRelativoBordeCentro.count())

Datos de posiciones, velocidades y velocidad angular relativa guardados en archivos CSV.


In [8]:
import matplotlib.pyplot as plt ###

# Ajuste de curva para aceleracion Y
def fit_acceleration_curve(df):
    def model_func(x, a):
        return np.full_like(x, a)

    x_data = df['Time (sec)'].values
    y_data = df['Acceleration_Y'].values

    popt, pcov = curve_fit(model_func, x_data, y_data)

    y_pred = model_func(x_data, *popt)
    ss_res = np.sum((y_data - y_pred) ** 2)
    ss_tot = np.sum((y_data - np.mean(y_data)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)

    #return popt, pcov, r_squared

    # GRAFICO DE EJEMPLO
    x_fit = np.linspace(min(x_data), max(x_data), 1000)
    y_fit = model_func(x_fit, *popt)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_data,y=y_data,mode='markers',name='Datos originales',marker=dict(size=8,opacity=0.6,color='blue')))
    fig.add_trace(go.Scatter(x=x_fit,y=y_fit,mode='lines',name=f'Valor constante = {popt[0]:.4f}',line=dict(color='red', width=2)))
    fig.update_layout(title=dict(text='Ajuste constante para datos de aceleración',x=0.5,xanchor='center'),xaxis_title='Tiempo (s)',yaxis_title='Aceleración Y',showlegend=True,legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01),template='plotly_white',hovermode='x unified')
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    fig.show()

    return popt, pcov, r_squared

# popt, pcov, r_squared = fit_acceleration_curve(dfCentro_acceleration[dfCentro_acceleration['Time (sec)'].between(0.05, 0.35)])
popt, pcov, r_squared = fit_acceleration_curve(dfCentro_acceleration)
print(popt)
print(pcov)
print(r_squared)

[0.12893272]
[[3.2209324]]
0.0


# Graficos

In [9]:
pio.renderers.default = 'vscode'  # 'browser' | 'vscode' | colab

In [10]:
# Elegir dataframe a graficar
dfG = dfCentro
dfG_speed = dfCentro_speed.copy()
dfG_acceleration = dfCentro_acceleration.copy()

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Suponiendo que tienes tus DataFrames cargados (dfG, dfG_speed, dfG_acceleration)

# Crear figura y ejes
fig = make_subplots(rows=4, cols=2, subplot_titles=("Trayectoria del Objeto", "Velocidad del Objeto", "Posición X / Tiempo", "Posición Y / Tiempo","Velocidad X / Tiempo", "Velocidad Y / Tiempo", "Aceleración X / Tiempo", "Aceleración Y / Tiempo"))

# Grafica la trayectoria
fig.update_xaxes(title_text="Posición X (m)", range=[0.35, 0.6], row=1, col=1, scaleanchor="y") # Relación 1:1 entre X e Y
fig.update_yaxes(title_text="Posición Y (m)", row=1, col=1)
fig.add_trace(go.Scatter(x=dfG['X'], y=dfG['Y'], mode='lines', marker=dict(color='blue')), row=1, col=1)

# Grafica la velocidad
fig.update_yaxes(title_text="Velocidad (m/s)", row=1, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=1, col=2)
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Speed'], mode='lines', marker=dict(color='red'), name='Velocidad Total'), row=1, col=2)

# Grafica la posición X / tiempo
fig.update_yaxes(title_text="Posición X (m)", range=[0.35,0.6], row=2, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=2, col=1)
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['X'], mode='lines', marker=dict(color='green')), row=2, col=1)

# Grafica la posición Y / tiempo
fig.update_yaxes(title_text="Posición Y (m)", row=2, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=2, col=2)
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['Y'], mode='lines', marker=dict(color='magenta')), row=2, col=2)

# Grafica la velocidad X / tiempo
fig.update_yaxes(title_text="Velocidad X (m/s)", range=[-0.5,0.5], row=3, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=3, col=1)
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Speed_X'], mode='lines', marker=dict(color='blue')), row=3, col=1)

# Grafica la velocidad Y / tiempo
fig.update_yaxes(title_text="Velocidad Y (m/s)", row=3, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=3, col=2)
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Speed_Y'], mode='lines', marker=dict(color='red')), row=3, col=2)

# Grafica la aceleración X / tiempo
fig.update_yaxes(title_text="Aceleración X (m/s^2)", row=4, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=4, col=1)
fig.add_trace(go.Scatter(x=dfG_acceleration['Time (sec)'], y=dfG_acceleration['Acceleration_X'], mode='lines', marker=dict(color='green')), row=4, col=1)

# Grafica la aceleración Y / tiempo
fig.update_yaxes(title_text="Aceleración Y (m/s^2)", row=4, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=4, col=2)
fig.add_trace(go.Scatter(x=dfG_acceleration['Time (sec)'], y=dfG_acceleration['Acceleration_Y'], mode='lines', marker=dict(color='purple')), row=4, col=2)

# Actualizar los títulos y ajustar el diseño
fig.update_layout(height=1000, width=1200, title_text="Análisis de Movimiento del Centro del Objeto", showlegend=False)

# Mostrar los gráficos
fig.show()

In [11]:
# Elegir dataframe a graficar
dfG = dfBorde
dfG_speed = dfBorde_speed.copy()
dfG_acceleration = dfBorde_acceleration.copy()

# Crear figura y ejes
# 5 filas, 2 columnas
fig = make_subplots(
    rows=5, cols=2,
    subplot_titles=(
        "Trayectoria del Objeto", "Velocidad del Objeto",
        "Posición X / Tiempo", "Posición Y / Tiempo",
        "Velocidad X / Tiempo", "Velocidad Y / Tiempo",
        "Aceleración X / Tiempo", "Aceleración Y / Tiempo",
        "Velocidad Angular / Tiempo", ""
    )
)

# Grafica la trayectoria
fig.update_yaxes(title_text="Posición Y (m)", row=1, col=1)
fig.update_xaxes(title_text="Posición X (m)", row=1, col=1, scaleanchor="y")
fig.add_trace(go.Scatter(x=dfG['X'], y=dfG['Y'], mode='lines', marker=dict(color='blue')), row=1, col=1)

# Grafica la velocidad
fig.update_yaxes(title_text="Velocidad Y (m/s)", row=1, col=2)
fig.update_xaxes(title_text="Velocidad X (m/s)", row=1, col=2)
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Speed'], mode='lines', marker=dict(color='red'), name='Velocidad Total'), row=1, col=2)

# Grafica la posición X en función del tiempo
fig.update_yaxes(title_text="Posición X (m)", row=2, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=2, col=1)
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['X'], mode='lines', marker=dict(color='green')), row=2, col=1)

# Grafica la posición Y / tiempo
fig.update_yaxes(title_text="Posición Y (m)", row=2, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=2, col=2)
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['Y'], mode='lines', marker=dict(color='magenta')), row=2, col=2)

# Grafica la velocidad X / tiempo
fig.update_yaxes(title_text="Velocidad X (m/s)", row=3, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=3, col=1)
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Speed_X'], mode='lines', marker=dict(color='blue')), row=3, col=1)

# Grafica la velocidad Y / tiempo
fig.update_yaxes(title_text="Velocidad Y (m/s)", row=3, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=3, col=2)
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Speed_Y'], mode='lines', marker=dict(color='red')), row=3, col=2)

# Grafica la aceleración X / tiempo
fig.update_yaxes(title_text="Aceleración X (m/s^2)", row=4, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=4, col=1)
fig.add_trace(go.Scatter(x=dfG_acceleration['Time (sec)'], y=dfG_acceleration['Acceleration_X'], mode='lines', marker=dict(color='green')), row=4, col=1)

# Grafica la aceleración Y / tiempo
fig.update_yaxes(title_text="Aceleración Y (m/s^2)", row=4, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=4, col=2)
fig.add_trace(go.Scatter(x=dfG_acceleration['Time (sec)'], y=dfG_acceleration['Acceleration_Y'], mode='lines', marker=dict(color='purple')), row=4, col=2)

# Grafica la velocidad angular
fig.update_yaxes(title_text="Velocidad Angular (rad/s)", row=5, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=5, col=1)
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['Angular_Velocity'], mode='lines', marker=dict(color='orange')), row=5, col=1)

# Actualizar los títulos y ajustar el diseño
fig.update_layout(height=1200, width=1200, title_text="Análisis de Movimiento del Borde del Objeto", showlegend=False)

# Mostrar los gráficos
fig.show()

In [12]:
# Elegir dataframe a graficar
dfG = dfRelativoBordeCentro
dfG_speed = dfRelativoBordeCentro_speed
dfG_acceleration = dfRelativoBordeCentro_acceleration

# Crear figura y ejes
# 3 filas, 2 columnas
fig = make_subplots(rows=4, cols=2, subplot_titles=("Posición X / Tiempo", "Posición Y / Tiempo",
                                                    "Velocidad X / Tiempo", "Velocidad Y / Tiempo",
                                                    "Aceleración X / Tiempo", "Aceleración Y / Tiempo",
                                                    "Velocidad Angular / Tiempo", "Aceleración Angular / Tiempo"))

# Grafica la posición X en funcion del tiempo
fig.update_yaxes(title_text="Posición X (m)", row=1, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=1, col=1)
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['Relative_Position_X'], mode='lines', marker=dict(color='green')), row=1, col=1)

# Grafica la posición Y / tiempo
fig.update_yaxes(title_text="Posición Y (m)", row=1, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=1, col=2)
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['Relative_Position_Y'], mode='lines', marker=dict(color='magenta')), row=1, col=2)

# Grafica la velocidad X / tiempo
fig.update_yaxes(title_text="Velocidad X (m/s)", row=2, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=2, col=1)
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Relative_Speed_X'], mode='lines', marker=dict(color='blue')), row=2, col=1)

# Grafica la velocidad Y / tiempo
fig.update_yaxes(title_text="Velocidad Y (m/s)", row=2, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=2, col=2)
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Relative_Speed_Y'], mode='lines', marker=dict(color='red')), row=2, col=2)

# Grafica la aceleración X / tiempo
fig.update_yaxes(title_text="Aceleración X (m/s^2)", row=3, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=3, col=1)
fig.add_trace(go.Scatter(x=dfG_acceleration['Time (sec)'], y=dfG_acceleration['Relative_Acceleration_X'], mode='lines', marker=dict(color='green')), row=3, col=1)

# Grafica la aceleración Y / tiempo
fig.update_yaxes(title_text="Aceleración Y (m/s^2)", row=3, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=3, col=2)
fig.add_trace(go.Scatter(x=dfG_acceleration['Time (sec)'], y=dfG_acceleration['Relative_Acceleration_Y'], mode='lines', marker=dict(color='purple')), row=3, col=2)

# Grafica la velocidad angular
fig.update_yaxes(title_text="Velocidad Angular (rad/s)", row=4, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=4, col=1)
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['Relative_Angular_Velocity'], mode='lines', marker=dict(color='orange')), row=4, col=1)

#Grafica la aceleración angular
fig.update_yaxes(title_text="Aceleración Angular (rad/s)", row=4, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=4, col=2)
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['Relative_Angular_Acceleration'], mode='lines', marker=dict(color='black')), row=4, col=2)

# Actualizar los títulos y ajustar el diseño
fig.update_layout(height=1000, width=1200, title_text="Análisis del movimiento relativo de ambos puntos", showlegend=False)

# Mostrar los gráficos
fig.show() # Si hay error acá, revisar el renderer

<h1>Análisis de la Dinámica

In [13]:

#Distancia del borde inferior al centro de masa del yoyo en el punto inicial
distancia_inf = 1589

#Distancia del borde superior al centro de masa del yoyo en el punto inicial
distancia_sup = 351

#Por ahora con masa constante
#Calcular la de tensión 
df_tension = pd.DataFrame()
g = 9.04562529
df_tension['tension'] = (masa_yoyo * (g)) + (masa_yoyo * dfCentro_acceleration['Acceleration_Y'] )
df_tension['time (sec)'] = dfCentro_acceleration['Time (sec)']
peso= masa_yoyo * g


#Calcular largo del hilo en función del tiempo
df_largo = pd.DataFrame()
df_largo['hilo'] = (distancia_inf * const_XY) - dfCentro['Y']
df_largo['time (sec)'] = dfCentro['Time (sec)']


largoDf =len(dfCentro) - 1


# Mostrar la primera y última fila del DataFrame
primer_valor = df_largo.head(1)
ultimo_valor = df_largo.tail(1)

# Visualizar
print( primer_valor)
print( ultimo_valor)


       hilo  time (sec)
0  0.006927         0.0
          hilo  time (sec)
1642  0.884604     6.82835


In [ ]:
#Función para hallar la masa en cada momento:
masa_inicial_yoyo = 0.02235  # kg (cuerpo + hilo)
largoHilo = 0.96   #//(metros)
masaHilo = 0.00065 #//(kg)
densidad_hilo = masaHilo / largoHilo 
df_masa = pd.DataFrame()      

df_masa['masa del yoyo'] = (masa_inicial_yoyo - (densidad_hilo * df_largo['hilo'])) 
print(df_masa) 


      masayoyo
0     0.022345
1     0.022345
2     0.022345
3     0.022345
4     0.022345
...        ...
1638  0.021750
1639  0.021750
1640  0.021751
1641  0.021751
1642  0.021751

[1643 rows x 1 columns]


In [17]:
#Elegir dataframe
dfGTension = df_tension

fig = make_subplots(rows=1, cols=1, subplot_titles=("tension/Tiempo"))

fig.update_yaxes(title_text="Tension (N)", row=1, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=1, col=1)
fig.add_trace(go.Scatter(x=dfGTension['time (sec)'], y=dfGTension['tension'], mode='lines', marker=dict(color='red'), name='tension'), row=1, col=1)


fig.update_layout(height=750, width=750, title_text="Análisis Dinámico", showlegend=False)

fig.show()

# Energia

In [18]:
dfEnergia = dfCentro[['Time (sec)']].copy()
dfEnergia['Masa'] = masa_yoyo
dfEnergia['Energia_Potencial'] = 0.5 * dfEnergia['Masa'] * 9.81 * dfCentro['Y']
dfEnergia['Energia_Cinetica_Traslacional'] = 0.5 * dfEnergia['Masa'] * dfCentro['Speed']**2
#dfEnergia['Energia_Cinetica_Traslacional'] = dfEnergia.loc[0, 'Energia_Potencial'] - dfEnergia['Energia_Potencial'] ### TEST
dfEnergia['Energia_Cinetica_Rotacional'] = 0.5 * 0.5 * dfEnergia['Masa'] * dfBorde['R']**2 * dfBorde['Angular_Velocity']**2
#dfEnergia['Energia_Cinetica_Rotacional'] = 0 ### TEST
dfEnergia['Energia_Mecanica'] = dfEnergia['Energia_Cinetica_Traslacional'] + dfEnergia['Energia_Cinetica_Rotacional'] + dfEnergia['Energia_Potencial']

dfEnergia.describe()

,Time (sec),Masa,Energia_Potencial,Energia_Cinetica_Traslacional,Energia_Cinetica_Rotacional,Energia_Mecanica
count,1643.000000,1.643000e+03,1643.000000,1643.000000,1.636000e+03,1636.000000
mean,3.414175,2.235000e-02,0.032013,0.039443,2.917468e-03,0.074234
std,1.972975,3.470503e-18,0.025574,0.061176,5.189227e-03,0.071475
min,0.000000,2.235000e-02,0.004440,0.000002,3.017805e-10,0.005667
25%,1.707087,2.235000e-02,0.012021,0.002553,8.548921e-05,0.024511
50%,3.414175,2.235000e-02,0.024145,0.015199,4.910553e-04,0.053778
75%,5.121262,2.235000e-02,0.044596,0.052452,2.819051e-03,0.104135
max,6.828350,2.235000e-02,0.104218,0.464868,3.052164e-02,0.503885


In [19]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=dfEnergia['Time (sec)'], y=dfEnergia['Energia_Cinetica_Traslacional'],mode='lines', name='Energía Cinética Traslacional'))
fig.add_trace(go.Scatter(x=dfEnergia['Time (sec)'], y=dfEnergia['Energia_Cinetica_Rotacional'],mode='lines', name='Energía Cinética Rotacional'))
fig.add_trace(go.Scatter(x=dfEnergia['Time (sec)'], y=dfEnergia['Energia_Potencial'],mode='lines', name='Energía Potencial'))
fig.add_trace(go.Scatter(x=dfEnergia['Time (sec)'], y=dfEnergia['Energia_Mecanica'],mode='lines', name='Energía Mecánica Total'))

fig.update_layout(title='Energías del YoYo en función del Tiempo',xaxis_title='Tiempo (sec)',yaxis_title='Energía (J)',template='plotly_dark')

fig.show()
